In [5]:
import os

from datasets import load_from_disk

save_dir = "/data_x/yjoonjang/PAPERS/Anchor-Embedding/DATA"
os.makedirs(save_dir, exist_ok=True)

ds_ko_path = "/data_x/EMBEDDING/DATA/SFT/v2-filtered-new/train_ko_250625_KURE-v1_hn32_final"
ds_en_path = "/data_x/EMBEDDING/DATA/SFT/v2-filtered-new/rlhn_remove_680k_processed"

ds_ko = load_from_disk(ds_ko_path)
ds_en = load_from_disk(ds_en_path)

print(ds_ko)
print(ds_en)

Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'negative_8', 'negative_9', 'negative_10', 'negative_11', 'negative_12', 'negative_13', 'negative_14', 'negative_15', 'negative_16', 'negative_17', 'negative_18', 'negative_19', 'negative_20', 'negative_21', 'negative_22', 'negative_23', 'negative_24', 'negative_25', 'negative_26', 'negative_27', 'negative_28', 'negative_29', 'negative_30', 'negative_31', 'negative_32', 'label'],
    num_rows: 1539300
})
Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7'],
    num_rows: 323622
})


In [6]:
ds_ko[0]

{'anchor': '서소문본관 합동소방훈련 중 전시동 광장에서는 무슨 훈련을 진행했어',
 'positive': '서소문본관 합동소방훈련 결과보고 추진근거 공공기관의 소방안전관리에 관한 규정 제14조(소방훈련 및 교육) 서소문본관 합동소방훈련 실시 계획(총무과-5506, 2021.5.21.) 소방훈련 개요 일시 : 2021.5.31.(월) 14:00 ~ 15:30 장소 : SeMA홀(심폐소생술 등 응급처치 교육), 전시동 광장(화재 발생시 소화 훈련) 참석자(33명) - 중부소방서(3명) : ************ - 서소문본관(30명) : *****************',
 'negative_1': '충남 예산소방서(서장 김오식) 지난 28일 예산상설시장에서 합동 소방훈련을 실시했다고 밝혔다. 이번 훈련은 화재 발생 시 막대한 인명 ᆞ 재산피해가 우려되고 불특정 다수가 이용하는 화재경계지구로 선정된 예산상설시장에서 진행, 상인회로 구성된 자위소방대의 초기 대처능력을 강화하고 신속한 진화와 인명 대피능력을 배양하고자 마련됐으며 시나리오 없는 무각본으로 시행했다. 주요 내용은시장상인회와 지역 의용소방대의 비상소화장치(호스릴소화전 등)을 이용한 자위소방활동소방서 선착대의 화재진압 및 인명구조활동후착대의 소화전 점령 및 경찰의 주요도로 통제선 설치훈련강평 및 비상소화장치의 효율적 사용에 대한 교육 등이다. 이주진 화재대책과장은 "이번 훈련으로 자위소방대의 초기 대처능력 강화를 위한 좋은 계기가 됐으며 시장상인회를 중심으로 자위소방대의 주인의식을 통한 자율 안전체계 구축에 힘을 써주시길 당부한다"고 말했다. 한편, 훈련 진행 시 코로나-19 감염방지를 위하여 방역지침을 준수하여 진행했다.',
 'negative_2': '세종시 도담동은 세종소방서와 도담동 행정복지센터에서 초기 재난대응 역량 강화를 위한 화재진압 합동소방훈련을 했다고 21일 밝혔다. 이번 훈련은 어진소방서, 동 직원 자위소방대 등 30여 명과 펌프차, 물탱크차 등 소방차량 5대가 동원돼 행정복지센터

In [10]:
# label 구조 확인: label[0] = positive score, label[1:] = negative scores
print(f"label length: {len(ds_ko[0]['label'])}")
print(f"label[0] (positive score): {ds_ko[0]['label'][0]}")
print(f"label[1:5] (negative scores 1~4): {ds_ko[0]['label'][1:5]}")
print(f"all labels: {ds_ko[0]['label']}")

label length: 32
label[0] (positive score): 5.985762119293213
label[1:5] (negative scores 1~4): [-3.1418838500976562, -3.010608673095703, -4.423274517059326, -4.063968181610107]
all labels: [5.985762119293213, -3.1418838500976562, -3.010608673095703, -4.423274517059326, -4.063968181610107, -4.855483055114746, -2.9714770317077637, -3.9965128898620605, -4.9142656326293945, -5.935125350952148, -4.217612266540527, -1.4096529483795166, -2.658928632736206, -6.488472938537598, -4.7502360343933105, -5.379757881164551, -3.8590211868286133, -6.3052144050598145, -2.010293960571289, -3.3314945697784424, -5.339186668395996, -5.97364616394043, -7.350090026855469, -4.4434027671813965, -5.227395057678223, -7.105632781982422, -7.7144036293029785, -3.766171455383301, -7.8543877601623535, -2.679638147354126, -6.569330215454102, -4.836891174316406]


In [11]:
from datasets import Dataset

NUM_NEG_TO_SELECT = 7
NUM_PROC = 32

def select_negatives(sample):
    labels = sample['label']
    positive_score = labels[0]
    neg_scores = labels[1:]

    # 조건: 1) 음수이고, 2) positive 점수를 넘지 않는 negative
    candidates = []
    for neg_idx, neg_score in enumerate(neg_scores):
        if neg_score < positive_score:
            candidates.append((neg_idx, neg_score))

    # 점수가 가장 높은 순서로 정렬 (hard negative)
    candidates.sort(key=lambda x: x[1], reverse=True)

    result = {
        'anchor': sample['anchor'],
        'positive': sample['positive'],
        'num_valid_negatives': len(candidates),
    }

    new_labels = [positive_score]
    for i in range(NUM_NEG_TO_SELECT):
        if i < len(candidates):
            neg_idx, neg_score = candidates[i]
            result[f'negative_{i+1}'] = sample[f'negative_{neg_idx+1}']
            new_labels.append(neg_score)
        else:
            result[f'negative_{i+1}'] = ""
            new_labels.append(0.0)

    result['label'] = new_labels
    return result

ds_processed = ds_ko.map(
    select_negatives,
    num_proc=NUM_PROC,
    remove_columns=ds_ko.column_names,
    desc="Selecting negatives",
)
print(ds_processed)

Selecting negatives (num_proc=32): 100%|██████████| 1539300/1539300 [02:48<00:00, 9154.18 examples/s]  


Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'label', 'num_valid_negatives'],
    num_rows: 1539300
})


In [12]:
# valid / insufficient 분리
ds_ko_filtered = ds_processed.filter(lambda x: x['num_valid_negatives'] >= NUM_NEG_TO_SELECT, num_proc=NUM_PROC)
ds_insufficient = ds_processed.filter(lambda x: x['num_valid_negatives'] < NUM_NEG_TO_SELECT, num_proc=NUM_PROC)

# num_valid_negatives 컬럼 제거
ds_ko_filtered = ds_ko_filtered.remove_columns(['num_valid_negatives'])

print(f"Valid samples (>= 7 negatives): {len(ds_ko_filtered)}")
print(f"Insufficient samples (< 7 negatives): {len(ds_insufficient)}")
print(ds_ko_filtered)
print(ds_ko_filtered[0])

Filter (num_proc=32): 100%|██████████| 1539300/1539300 [00:05<00:00, 285616.60 examples/s]

Valid samples (>= 7 negatives): 1194002
Insufficient samples (< 7 negatives): 345298
Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7', 'label'],
    num_rows: 1194002
})
{'anchor': '서소문본관 합동소방훈련 중 전시동 광장에서는 무슨 훈련을 진행했어', 'positive': '서소문본관 합동소방훈련 결과보고 추진근거 공공기관의 소방안전관리에 관한 규정 제14조(소방훈련 및 교육) 서소문본관 합동소방훈련 실시 계획(총무과-5506, 2021.5.21.) 소방훈련 개요 일시 : 2021.5.31.(월) 14:00 ~ 15:30 장소 : SeMA홀(심폐소생술 등 응급처치 교육), 전시동 광장(화재 발생시 소화 훈련) 참석자(33명) - 중부소방서(3명) : ************ - 서소문본관(30명) : *****************', 'negative_1': '충남 공주소방서가 코로나-19 백신 예방접종센터로 지정된 백제체육관에서 자위소방대 중심 합동 소방 훈련을 시행했다고 9일 밝혔다. 주요 훈련은 통보 연락(화재 발견 및 전파) 초기 대응(초기 소화 및 안전조치) 피난(피난 개시 명령, 피난 절차 및 방법) 인명 검색 및 화재 진압 연소 확대 방지 활동 구조.구급활동 백신 접종 물품 안전 반출 등 실제 화재 발생 상황을 가정해 무각본 소방 훈련을 진행했다. 류석윤 소방서장은 "코로나19 종식을 위한 발판이 될 백신 접종이 안전하게 진행될 수 있도록 만일의 재난 상황에 철저히 대비해 나가겠다"고 말했다.', 'negative_2': '김부겸 장관, 고층 건축물 대형 화재 대응 훈련 현장 점검 한편, 훈련 현장에서는 70m굴절탑차, 무인방수탑차, 소방헬

In [13]:
# 기준을 충족하지 못한 샘플들 확인
import pandas as pd

if len(ds_insufficient) > 0:
    df_insufficient = ds_insufficient.to_pandas()
    print(f"Insufficient samples: {len(df_insufficient)}")
    print(f"\nnum_valid_negatives 분포:")
    print(df_insufficient['num_valid_negatives'].value_counts().sort_index())
    print(f"\n--- 처음 10개 샘플 ---")
    for i in range(min(10, len(ds_insufficient))):
        row = ds_insufficient[i]
        print(f"\n[Sample {i}]")
        print(f"  anchor: {row['anchor'][:100]}...")
        print(f"  positive_score: {row['label'][0]:.4f}")
        print(f"  valid negatives: {row['num_valid_negatives']}")
        print(f"  neg scores: {row['label'][1:]}")
else:
    print("모든 샘플이 7개의 negative를 충족합니다!")

Insufficient samples: 345298

num_valid_negatives 분포:
num_valid_negatives
0    61924
1    53572
2    49695
3    47074
4    45545
5    44180
6    43308
Name: count, dtype: int64

--- 처음 10개 샘플 ---

[Sample 0]
  anchor: 미디어재단 티비에스 개인정보 처리지침 상 홈페이지 회원정보의 보유기간은 얼마나 돼...
  positive_score: -10.0661
  valid negatives: 1
  neg scores: [-11.513659477233887, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

[Sample 1]
  anchor: 143,100,000원을 연구원 연구사업수행규칙에 따라서 분담해야 하는 곳은 어디야...
  positive_score: -4.3047
  valid negatives: 3
  neg scores: [-4.696891784667969, -4.765977382659912, -5.331151962280273, 0.0, 0.0, 0.0, 0.0]

[Sample 2]
  anchor: 2021년 6월 14일에 지출한 의정활동 수행에 따른 소요 경비는 무엇에 관한 지출일까...
  positive_score: -4.8385
  valid negatives: 5
  neg scores: [-7.517881870269775, -7.57780647277832, -7.7768168449401855, -8.124103546142578, -8.981027603149414, 0.0, 0.0]

[Sample 3]
  anchor: 구의회사무국 직원 2021년 5월분 시간외근무수당을 어떻게 지급할 예정이야...
  positive_score: -10.8450
  valid negatives: 3
  neg scores: [-11.672364234924316, -11.9661

In [14]:
# 저장
ds_ko_filtered.save_to_disk(os.path.join(save_dir, "train_ko_filtered"))
print(f"Saved to {os.path.join(save_dir, 'ds_ko_filtered')}")

Saving the dataset (27/27 shards): 100%|██████████| 1194002/1194002 [02:50<00:00, 6983.63 examples/s] 

Saved to /data_x/yjoonjang/PAPERS/Anchor-Embedding/DATA/ds_ko_filtered


In [17]:
ds_ko_filtered = ds_ko_filtered.remove_columns(["label"])
ds_ko_filtered

Dataset({
    features: ['anchor', 'positive', 'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5', 'negative_6', 'negative_7'],
    num_rows: 1194002
})

In [18]:
from datasets import concatenate_datasets
ds_ko_en_filtered = concatenate_datasets([ds_ko_filtered, ds_en])
ds_ko_en_filtered = ds_ko_en_filtered.shuffle(seed=42)
ds_ko_en_filtered.save_to_disk(os.path.join(save_dir, "train_ko_en_filtered"))
print(f"Saved to {os.path.join(save_dir, 'train_ko_en_filtered')}")

Saving the dataset (31/31 shards): 100%|██████████| 1517624/1517624 [03:13<00:00, 7857.27 examples/s] 

Saved to /data_x/yjoonjang/PAPERS/Anchor-Embedding/DATA/train_ko_en_filtered
